In [1]:
!pip install pytorch_lightning

     |████████████████████████████████| 256kB 2.6MB/s 
     |████████████████████████████████| 829kB 7.7MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=aa38e58fa4acacf8a053e68bdcf345c4b7004428c194d01132e3d3b8e304434c
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [2]:
# Mount your personal google drive
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
primary_directory = '/content/drive/My Drive/KBP_Challenge'
%cd /content/drive/My\ Drive/KBP_Challenge
%ls

/content/drive/My Drive/KBP_Challenge
checkpoints/  hparams.yaml     options/        src/
data/         lightning_logs/  provided_code/


In [0]:
import pytorch_lightning as pl
from src.pix2pix_model import Pix2PixModel
from src.kbp_dataset import KBPDataset
from options.train_options import TrainOptions
from provided_code.general_functions import get_paths
from provided_code.dose_evaluation_class import EvaluateDose
import numpy as np

In [5]:
args = ['--batchSize', '8',
        '--which_model_netG', 'unet_128_3d',
        '--which_direction', 'AtoB',
        '--input_nc', '1',
        '--lambda_A', '100',
        '--lr_policy', 'step',
        '--epoch_count', '50',
        '--lr_decay_iters', '100',
        '--lr', '0.0001']

opt = TrainOptions().parse(args)

------------ Options -------------
batchSize: 8
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
dataroot: None
dataset_mode: unaligned
display_freq: 200
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 128
epoch_count: 50
fineSize: 128
gpu_ids: []
init_type: normal
input_nc: 1
isTrain: True
lambda_A: 100.0
lambda_B: 10.0
lambda_identity: 0.5
loadSize: 128
lr: 0.0001
lr_decay_iters: 100
lr_policy: step
max_dataset_size: inf
model: pix2pix
nThreads: 2
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
niter: 100
niter_decay: 100
no_dropout: False
no_flip: False
no_html: False
no_img: False
no_lsgan: False
norm: batch_3d
nwf: 64
output_nc: 1
phase: train
pool_size: 50
print_freq: 200
recon_loss: l1
resize_or_crop: resize_and_crop
save_epoch_freq: 5
save_latest_freq: 5000
serial_batches: False
update_html_freq: 1000
wasserstein: False
which_direction: AtoB
which_epoch: latest
which_model_netD: n_layers_3d
which_model_netG: unet_128_3d
which_mo

In [0]:
checkpoint_number = 74
checkpoint_file = '{}/checkpoints/epoch={}.ckpt'.format(primary_directory, checkpoint_number)
# checkpoint_file = '{}/checkpoints/test.ckpt'.format(primary_directory)
dataset_dir = '{}/data'.format(primary_directory)
model = Pix2PixModel.load_from_checkpoint(checkpoint_file, opt, dataset_dir, stage='hold-out-tests')

# print(model.state_dict())

trainer = pl.Trainer(max_epochs=1)
trainer.test(model)

initialization method [normal]
initialization method [normal]


GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
No environment variable for node rank defined. Set as 0.
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You passed  in a `test_dataloader` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnin

In [0]:
training_data_dir = '{}/train-pats'.format(dataset_dir)

plan_paths = get_paths(training_data_dir, ext='')  # gets the path of each plan's directory
num_train_pats = np.minimum(100, len(plan_paths))  # number of plans that will be used to train model
hold_out_paths = plan_paths[num_train_pats:]  # list of paths used for held out testing
data_loader_hold_out_eval = KBPDataset(hold_out_paths, mode_name='evaluation')

prediction_dir = '{}/data/results/pix2pix_default/hold-out-tests-predictions'.format(primary_directory)
prediction_paths = get_paths(prediction_dir, ext='csv')
hold_out_prediction_loader = KBPDataset(prediction_paths, mode_name='predicted_dose')

dose_evaluator = EvaluateDose(data_loader_hold_out_eval, hold_out_prediction_loader)

if not data_loader_hold_out_eval.file_paths_list:
    print('No patient information was given to calculate metrics')
else:
    dvh_score, dose_score = dose_evaluator.make_metrics()
    print('For this out-of-sample test:\n'
          '\tthe DVH score is {:.3f}\n '
          '\tthe dose score is {:.3f}'.format(dvh_score, dose_score))
